In [2]:
import numpy as np

In [3]:
random_generator = np.random.default_rng()
def generate_data(n_feature, n_values):
    features = random_generator.random((n_feature, n_values))
    targets = random_generator.random((n_feature))
    return features, targets

In [4]:
trainX, trainY = generate_data(500, 4)
print(trainX, trainY)
print(trainX.shape)
print(trainY.shape)
testX, testY = generate_data(5, 4)

[[0.94928944 0.84073432 0.37816697 0.31453758]
 [0.46060153 0.3044845  0.03329424 0.66183352]
 [0.58753424 0.34354153 0.46066276 0.53741502]
 ...
 [0.63914544 0.30926539 0.28508588 0.76478394]
 [0.96394157 0.93198208 0.18049051 0.36080432]
 [0.25908904 0.83322895 0.8703505  0.25411898]] [0.79717022 0.29297032 0.32208494 0.33153102 0.92302275 0.40745805
 0.0593008  0.48239436 0.88636914 0.2100528  0.74835015 0.33977656
 0.65253338 0.70576378 0.4443627  0.95048088 0.16289468 0.57858
 0.36428965 0.30518627 0.24781828 0.58727904 0.53882229 0.58253411
 0.71018687 0.8707764  0.10548362 0.52686837 0.21720308 0.94530492
 0.90348674 0.18712822 0.76390693 0.26260302 0.63891779 0.35477584
 0.14349842 0.21822285 0.21964746 0.50597377 0.80417401 0.27262006
 0.28773968 0.51200321 0.92354464 0.71152338 0.11730996 0.18522249
 0.72929865 0.42040461 0.44751511 0.67486169 0.90468853 0.48966297
 0.07647041 0.98163107 0.99148102 0.70230576 0.44483709 0.7263314
 0.51577542 0.49151163 0.45038953 0.71261555 0

In [8]:
class RNN:
    def __init__(self):
        self.global_weights = [1,1] # [input, recurrent weight]
        self.local_weights = [0.001, 0.001] # Why two values? Because RNN Model used to 2 value(hidden state of previeous node, input value of current node)
        self.W_sign = [0,0] # 부정적인 예측의 경우 RNN을 조정하기 위함.

        # 예상시간을 찾기위해 사용.
        self.eta_p = 1.2
        self.eta_n = 0.5
    
    # State Handler roles to accept input value 'x' and last state of previous node
    def state_handler(self, input_x, previous_state):
        return input_x * self.global_weights[0] + previous_state * self.global_weights[1]

    def forward_propagation(self, x):
        # Computes the forward propagation of the RNN.
        S = np.zeros((x.shape[0], x.shape[1]+1))
        for k in range(0, x.shape[1]):
            next_state = self.state_handler(x[:,k], S[:,k])
            S[:,k+1] = next_state
        return S

    def backward_propagation(self, x, s, grad_out):
        # Computes the backward proopagation of the RNN.
        grad_over_time = np.zeros((x.shape[0], x.shape[1]+1))
        grad_over_time[:,-1] = grad_out

        wx_grad = 0
        wy_grad = 0
        for k in range(x.shape[1], 0, -1):
            wx_grad += np.sum(grad_over_time[:, k] * x[:, k-1])
            wy_grad += np.sum(grad_over_time[:, k] * s[:, k-1])

            grad_over_time[:, k-1] = grad_over_time[:,k]*self.global_weights[1]
        return (wx_grad, wy_grad), grad_over_time

    def update_rprop(self, x, y, w_prev_sign, local_weight):
        s = self.forward_propagation(x)
        grad_out = 2 * (s[:, -1] - y) / 500
        W_grads, _ = self.backward_propagation(x, s, grad_out)
        self.W_sign = np.sign(W_grads)

        for i, _ in enumerate(self.global_weights):
            if self.W_sign[i] == w_prev_sign[i]:
                local_weight[i] *= self.eta_p
            else:
                local_weight[i] *= self.eta_n
        self.local_weights = local_weight
    
    def train(self, x, y, training_epochs):
        for epochs in range(training_epochs):
            self.update_rprop(x, y, self.W_sign, self.local_weights)

            for i, _ in enumerate(self.global_weights):
                self.global_weights[i] -= self.W_sign[i] * self.local_weights[i]

In [9]:
rnn = RNN()
rnn.train(trainX, trainY, 200)

print(f"Targets are: {testY}")
y = rnn.forward_propagation(testX)[:, -1]
print(f"Predicted are: {y}")

Targets are: [0.33685009 0.69921009 0.22343483 0.22313232 0.00506686]
Predicted are: [0.32688468 0.34737757 0.75924203 0.66266829 0.65070703]
